## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-12-05-08-50-26 +0000,bbc,What are freeze-dried sweets and why are they ...,https://www.bbc.com/news/articles/czdr8r7d7vmo...
1,2025-12-05-08-50-00 +0000,bbc,Government pledges to end children living in B&Bs,https://www.bbc.com/news/articles/c0l9nrkd4z6o...
2,2025-12-05-08-47-00 +0000,wsj,Euro Could Benefit From Lower Dollar Hedging C...,https://www.wsj.com/finance/currencies/singapo...
3,2025-12-05-08-46-10 +0000,nypost,US and Kenya sign first of what are expected t...,https://nypost.com/2025/12/05/world-news/us-an...
4,2025-12-05-08-40-00 +0000,wsj,Equinor Makes North Sea Gas Discoveries,https://www.wsj.com/business/energy-oil/equino...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2354/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
255,trump,48
328,new,20
708,he,13
152,ukraine,13
193,gaza,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
228,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...,122
135,2025-12-04-23-23-00 +0000,wsj,The Trump administration intends to formally s...,https://www.wsj.com/politics/policy/white-hous...,119
371,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...,108
117,2025-12-04-23-59-36 +0000,bbc,Trump hires new architect for White House ball...,https://www.bbc.com/news/articles/cjdrzd41j9po...,92
106,2025-12-05-00-21-26 +0000,nypost,Trump sees peace ‘eventually’ in Ukraine after...,https://nypost.com/2025/12/04/world-news/trump...,90


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
228,122,2025-12-04-19-19-46 +0000,nypost,Accused DC pipe-bomber Brian Cole Jr.’s nabbed...,https://nypost.com/2025/12/04/us-news/accused-...
105,81,2025-12-05-00-22-00 +0000,wsj,"In a briefing for lawmakers, Navy Adm. Frank “...",https://www.wsj.com/politics/national-security...
371,55,2025-12-04-10-00-00 +0000,wsj,Sen. Elizabeth Warren and other Democrats are ...,https://www.wsj.com/politics/policy/democrats-...
358,54,2025-12-04-10-30-10 +0000,nypost,Putin calls talks with US on ending Ukraine wa...,https://nypost.com/2025/12/04/world-news/putin...
307,49,2025-12-04-14-37-28 +0000,nypost,"Netanyahu taunts Zohran Mamdani, saying he’ll ...",https://nypost.com/2025/12/04/us-news/benjamin...
81,44,2025-12-05-01-21-23 +0000,nypost,Father of 14-year-old killed in Stockton shoot...,https://nypost.com/2025/12/04/us-news/father-o...
3,43,2025-12-05-08-46-10 +0000,nypost,US and Kenya sign first of what are expected t...,https://nypost.com/2025/12/05/world-news/us-an...
128,39,2025-12-04-23-42-38 +0000,latimes,"Supreme Court rules for Texas Republicans, all...",https://www.latimes.com/politics/story/2025-12...
200,37,2025-12-04-21-11-38 +0000,nypost,Trump boasts heads of Rwanda and Congo will be...,https://nypost.com/2025/12/04/us-news/trump-bo...
140,36,2025-12-04-23-11-43 +0000,nypost,Federal grand jury declines to indict NY AG Le...,https://nypost.com/2025/12/04/us-news/federal-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
